In [1]:
!pip install keras

    100% |################################| 307kB 3.2MB/s ta 0:00:01
    100% |################################| 51kB 6.8MB/s ta 0:00:011
    100% |################################| 276kB 3.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built pyyaml
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 38s 0us/step


In [3]:
# load pre-trained conv net base
img_rows, img_cols, channels = 96, 96, 3

from keras.applications.vgg16 import VGG16, preprocess_input
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channels))

58892288/58889256 [==============================] - 4s 0us/step


In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [5]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) 

In [5]:
vgg_model.trainable = False

In [7]:
# reshape images dimension to 96, 96, 3
import scipy.misc
x_train_big = np.array([scipy.misc.imresize(img, (img_rows, img_cols, channels))
                            for img in x_train]).astype('float32')

x_train_big /= 255.0
# vgg16_input_train = preprocess_input(x_train_big)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.


In [8]:
datagen.fit(x_train_big)

In [9]:
vgg_model.trainable = True
set_trainable = False
for layer in vgg_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
len(vgg_model.trainable_weights)

6

In [11]:
# one-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
## Model building
import keras.optimizers
model = models.Sequential()
model.add(vgg_model)
model.add(layers.Conv2D(filters=256, kernel_size=2))
model.add(layers.Dropout(0.4))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.4))    
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-5),
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

In [13]:
filepath="vgg_cifar10_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='max')

callbacks_list = [checkpoint]

In [19]:
history_callback = model.fit_generator(datagen.flow(x_train_big, y_train,
                                     batch_size=32),
                        samples_per_epoch=x_train_big.shape[0],
                        nb_epoch=50, validation_data=(x_test_big, y_test), callbacks=callbacks_list, verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=50, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=1562, verbose=1)`
  after removing the cwd from sys.path.


Epoch 1/50
1562/1562 [==============================] - 171s 110ms/step - loss: 0.5946 - acc: 0.7960 - val_loss: 0.4909 - val_acc: 0.8346

Epoch 00001: val_acc improved from 0.62150 to 0.83460, saving model to vgg_cifar10_weights.hdf5
Epoch 2/50
1562/1562 [==============================] - 171s 110ms/step - loss: 0.5000 - acc: 0.8291 - val_loss: 0.4885 - val_acc: 0.8408

Epoch 00002: val_acc improved from 0.83460 to 0.84080, saving model to vgg_cifar10_weights.hdf5
Epoch 3/50
1562/1562 [==============================] - 171s 110ms/step - loss: 0.4428 - acc: 0.8504 - val_loss: 0.4746 - val_acc: 0.8422

Epoch 00003: val_acc improved from 0.84080 to 0.84220, saving model to vgg_cifar10_weights.hdf5
Epoch 4/50
1562/1562 [==============================] - 171s 110ms/step - loss: 0.3949 - acc: 0.8653 - val_loss: 0.4436 - val_acc: 0.8570

Epoch 00004: val_acc improved from 0.84220 to 0.85700, saving model to vgg_cifar10_weights.hdf5
Epoch 5/50
1562/1562 [==============================] - 171s

KeyboardInterrupt: 

In [10]:
## Training
history = model.fit(vgg16_input_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 127s 3ms/step - loss: 1.7359 - acc: 0.5107 - val_loss: 0.7201 - val_acc: 0.7695
Epoch 2/10
40000/40000 [==============================] - 125s 3ms/step - loss: 0.6924 - acc: 0.7842 - val_loss: 0.5400 - val_acc: 0.8301
Epoch 3/10
40000/40000 [==============================] - 125s 3ms/step - loss: 0.4743 - acc: 0.8529 - val_loss: 0.4488 - val_acc: 0.8646
Epoch 4/10
40000/40000 [==============================] - 125s 3ms/step - loss: 0.3428 - acc: 0.8919 - val_loss: 0.4378 - val_acc: 0.8699
Epoch 5/10
40000/40000 [==============================] - 125s 3ms/step - loss: 0.2500 - acc: 0.9202 - val_loss: 0.4263 - val_acc: 0.8785
Epoch 6/10
40000/40000 [==============================] - 125s 3ms/step - loss: 0.1829 - acc: 0.9405 - val_loss: 0.4481 - val_acc: 0.8848
Epoch 7/10
40000/40000 [==============================] - 125s 3ms/step - loss: 0.1355 - acc: 0.9563 - val_loss: 0.4823 - 

KeyboardInterrupt: 

In [18]:
x_test_big = np.array([scipy.misc.imresize(img, (img_rows, img_cols, channels))
                            for img in x_test]).astype('float32')

x_test_big /= 225.0
# x_test_big = preprocess_input(x_test_big)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [20]:
loss, accuracy = model.evaluate(x_test_big, y_test)
print("Current model gives accuracy of {acc} %".format(acc=accuracy * 100))

10000/10000 [==============================] - 22s 2ms/step
Current model gives accuracy of 87.16000000000001 %


In [13]:
from helper_functions.plot_history import plot_history

In [14]:
plot_history(history)

NameError: name 'history' is not defined